# Steganografia - testy narzędzi
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia - testy narzędzi/sprawozdanie.md](https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.md)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł
- Anton Maisiuk

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - Ocena narzędzi steganograficznych 

Na podstawie listy ze strony [Wikipedii](https://en.wikipedia.org/wiki/Steganography_tools) wybierz 5 narzędzi steganograficznych i dokonaj ich oceny (badań):
- porównaj szybkość działania dla tego samego pliku,
- jeżeli to możliwe, to oceń ilość/jakość zmian pliku źródłowego.


## Zadanie 2 - Własny system steganograficzny

Wykonaj prosty system steganograficzny plików graficznych BMP polegający na osadzaniu w najmniej znaczących bitach danego stegotekstu:
- ilość bitów ma być zmienna, od 1 do 4,
- dokonaj oceny wizualnej głębokości zmian pliku źródłowego w zależności od ilości bitów modyfikowanych.